In [ ]:
from ultralytics import YOLO
import cv2

# โหลดโมเดล
model = YOLO('bestv8.pt')
print("Class names:", model.names)

# เปิดวิดีโอ
cap = cv2.VideoCapture("automation solution.mp4")

frame_size = (640, 640)  # ปรับขนาดให้ตรงกับตอนเทรน ถ้าเทรน 640x640

# กำหนดจุดเส้นเฉียง
mid_x = frame_size[0] // 2
new_x = mid_x + 200  # เลื่อนเส้นไปทางขวา
line_start = (new_x, 0)
line_end = (new_x - 300, frame_size[1])

def is_left_side(px, py, line_start, line_end):
    x1, y1 = line_start
    x2, y2 = line_end
    # สูตร cross product เพื่อตรวจว่าจุดอยู่ซ้ายหรือขวาของเส้น
    return (x2 - x1)*(py - y1) - (y2 - y1)*(px - x1) > 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize ให้ตรงกับขนาดเทรน
    resized_frame = cv2.resize(frame, frame_size)

    # รัน YOLO ด้วย confidence ต่ำลง
    results = model(resized_frame, conf=0.05)

    boxes = results[0].boxes.xyxy.cpu().numpy() if results[0].boxes.xyxy is not None else []
    class_ids = results[0].boxes.cls.cpu().numpy() if results[0].boxes.cls is not None else []

    print(f"Boxes detected: {len(boxes)}")
    print("Class IDs:", class_ids)

    left_objects = []
    right_objects = []

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2

        # ตรวจว่ากลางวัตถุอยู่ซ้ายหรือขวา ตามเส้นเฉียง
        if is_left_side(center_x, center_y, line_start, line_end):
            left_objects.append(box)
        else:
            right_objects.append(box)

        class_id = int(class_ids[i])
        label = model.names.get(class_id, str(class_id))

        # วาดกรอบ + label
        cv2.rectangle(resized_frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
        cv2.putText(resized_frame, f'{label}', (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # ตรวจชน
    for l in left_objects:
        for r in right_objects:
            lx2 = l[2]
            rx1 = r[0]
            gap = rx1 - lx2
            if 0 < gap < 20:
                cv2.putText(resized_frame, "STOP LEFT CONVEYOR", (50, 65),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)

    # วาดเส้นแบ่งเฉียงที่เลื่อนไปทางขวา
    cv2.line(resized_frame, line_start, line_end, (0, 255, 255), 2)

    # ปรับตำแหน่งข้อความให้เหมาะสมกับเส้นใหม่
    cv2.putText(resized_frame, "LEFT", (50, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    cv2.putText(resized_frame, "RIGHT", (new_x + 60, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # แสดงจำนวนวัตถุ
    cv2.putText(resized_frame, f"Left: {len(left_objects)}", (50, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
    cv2.putText(resized_frame, f"Right: {len(right_objects)}", (50, 130),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

    cv2.imshow("Conveyor Simulation", resized_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()